In [1]:
import tensorflow as tf
from tensorflow import keras

import pathlib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset_url = "./dataset brasileiro/TREINO"
data_dir = keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
data_dir = pathlib.Path(data_dir)

batch_size = 32
img_height = 180
img_width = 180

dataset_treino = keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

dataset_validacao = keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [3]:
class_names = dataset_treino.class_names
num_classes = len(class_names)
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [4]:
model = keras.models.Sequential([
  keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(num_classes, activation = 'softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
history = model.fit(dataset_treino, validation_data=dataset_validacao, epochs=5)

Epoch 1/5
92/92 [==============================] - 6s 28ms/step - loss: 1.3922 - accuracy: 0.4101 - val_loss: 1.1397 - val_accuracy: 0.5000
Epoch 2/5
92/92 [==============================] - 2s 20ms/step - loss: 1.0240 - accuracy: 0.5903 - val_loss: 0.9787 - val_accuracy: 0.6049
Epoch 3/5
92/92 [==============================] - 2s 21ms/step - loss: 0.8158 - accuracy: 0.6931 - val_loss: 0.9593 - val_accuracy: 0.6267
Epoch 4/5
92/92 [==============================] - 2s 21ms/step - loss: 0.6272 - accuracy: 0.7660 - val_loss: 0.9658 - val_accuracy: 0.6526
Epoch 5/5
92/92 [==============================] - 2s 21ms/step - loss: 0.4330 - accuracy: 0.8437 - val_loss: 1.1461 - val_accuracy: 0.6417


In [8]:
# Carregamento da base de teste
test_dataset_url = "./dataset brasileiro"
test_data_dir = keras.utils.get_file('TEST', origin=test_dataset_url, untar=True)
test_data_dir = pathlib.Path(test_data_dir)

dataset_teste = keras.utils.image_dataset_from_directory(
  test_data_dir,
  image_size=(img_height, img_width),
  batch_size=batch_size)

ValueError: unknown url type: './dataset brasileiro'

In [ ]:
# Classificação e acurácia
loss, accuracy = model.evaluate(dataset_teste)
print('Acurácia da base de teste:', accuracy)

In [ ]:
# Salvar modelo
model.save('modelo.keras')

In [ ]:
# Carregar modelo
model_carregado = keras.models.load_model('modelo.keras')

model_carregado.summary()

In [ ]:
# Classificar exemplo
sunflower_url = "modelo.keras"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.utils.load_img(sunflower_path, target_size=(img_height, img_width))
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

In [ ]:
predictions = model_carregado.predict(img_array)

score = predictions[0]

print(
    "Classificado como {} com {:.2f}% de confiança."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)